<a href="https://colab.research.google.com/github/abinbena/SYNRGY6/blob/main/model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.regularizers import l1_l2
from tensorflow.keras.applications import MobileNet, EfficientNetB0, Xception

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
train_df = pd.read_csv("/content/drive/My Drive/dataset_V1/train/label.csv")
test_df = pd.read_csv("/content/drive/My Drive/dataset_V1/test/label.csv")
val_df = pd.read_csv("/content/drive/My Drive/dataset_V1/valid/label.csv")

In [31]:
# Tentukan jalur ke direktori gambar
train_dir = "/content/drive/My Drive/dataset_V1/train"
test_dir = "/content/drive/My Drive/dataset_V1/test"
val_dir = "/content/drive/My Drive/dataset_V1/valid"

In [32]:
# Definisikan generator gambar untuk data pelatihan dan validasi
train_datagen = ImageDataGenerator(rescale =1./255,
                                   rotation_range =20,
                                   brightness_range=[0.8, 1.2],
                                   horizontal_flip =True,
                                   vertical_flip=True,
                                   shear_range = 0.2,
                                   fill_mode = "nearest")

validation_datagen = ImageDataGenerator(rescale=1./255)

In [33]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = train_dir,
    x_col='filename',
    y_col='class',
    target_size=(320, 320),
    batch_size=64,
    class_mode='categorical'
)

Found 4200 validated image filenames belonging to 4 classes.


In [34]:
validation_generator = validation_datagen.flow_from_dataframe(
    val_df,
    directory = val_dir,
    x_col='filename',
    y_col='class',
    target_size=(320, 320),
    batch_size=64,
    class_mode='categorical'
)

Found 407 validated image filenames belonging to 4 classes.


In [ ]:
# @title MobileNet
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

# Membuat regularizer ElasticNet
elasticnet_regularizer = l1_l2(l1=0.01, l2=0.01)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation ="relu", kernel_regularizer=elasticnet_regularizer),
    BatchNormalization(),
    Dropout(0.1),
    Dense(4, activation ="softmax")
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
# @title EfficientNet
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

# Membuat regularizer ElasticNet
elasticnet_regularizer = l1_l2(l1=0.01, l2=0.01)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation ="relu", kernel_regularizer=elasticnet_regularizer),
    BatchNormalization(),
    Dropout(0.1),
    Dense(4, activation ="softmax")
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
# @title Xception
base_model = Xception(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

# Membuat regularizer ElasticNet
elasticnet_regularizer = l1_l2(l1=0.01, l2=0.01)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation ="relu", kernel_regularizer=elasticnet_regularizer),
    BatchNormalization(),
    Dropout(0.1),
    Dense(4, activation ="softmax")
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

83683744/83683744 [==============================] - 0s 0us/step


In [36]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 flatten_3 (Flatten)         (None, 204800)            0         
                                                                 
 dense_6 (Dense)             (None, 1024)              209716224 
                                                                 
 batch_normalization_7 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 4)                 4100      
                                                      

In [37]:
# Checkpoint callback
checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/dataset_V1/best_model.h5',
                             monitor='val_accuracy',
                             mode='max',
                             verbose = 1,
                             save_best_only=True)

In [38]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=1,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint]
)

 3/66 [>.............................] - ETA: 57:52 - loss: 4200.2202 - accuracy: 0.5469  

KeyboardInterrupt: 

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

# Plot training & validation accuracy values
axs[0].plot(history.history['accuracy'], label='Training Accuracy')
axs[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axs[0].set_title('Model Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].legend(loc='lower right')

# Plot training & validation loss values
axs[1].plot(history.history['loss'], label='Training Loss')
axs[1].plot(history.history['val_loss'], label='Validation Loss')
axs[1].set_title('Model Loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].legend(loc='upper right')

plt.tight_layout()

plt.show()

In [ ]:
model.save('/content/drive/My Drive/dataset_V1/cnn_model.h5')

In [ ]:
model = load_model("/content/drive/My Drive/dataset_V1/best_model.h5")
#best_model = load_model("best_model.h5")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory = test_dir,
    x_col='filename',
    y_col='class',
    target_size=(320, 320), #editable (pastikan sama dengan input shape/target shape pada training sebelumnya)
    batch_size=1, #editable
    class_mode='categorical'
)

Found 206 validated image filenames belonging to 4 classes.


In [ ]:
evaluation1 = model.evaluate_generator(test_generator, steps=len(test_generator))

<ipython-input-13-025c0bf545e9>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation1 = model.evaluate_generator(test_generator, steps=len(test_generator))


In [ ]:
# Print hasil evaluasi
print("Latest Model")
print("Loss:", evaluation1[0])
print("Accuracy:", evaluation1[1])

#print("Best Model")
#print("Loss:", evaluation2[0])
#print("Accuracy:", evaluation2[1])

Latest Model
Loss: 1.1966100931167603
Accuracy: 0.8009708523750305
